# 3. 単語のベクトル表現

## 3.1. 準備

In [ ]:
!pip install janome

from janome.tokenizer import Tokenizer
import re
import torchtext

j_t = Tokenizer()

# 単語分割の関数
def tokenizer_janome(text):
    return [token for token in j_t.tokenize(text, wakati=True)]

# 前処理
def preprocessing_text(text):
    text = re.sub("\r", "", text)  # 改行の削除
    text = re.sub("\n", "", text)  # 改行の削除
    text = re.sub("　", "", text)  # 全角スペースの削除
    text = re.sub(" ", "", text)   # 半角スペースの削除
    text = re.sub(r"[0-9 ０-９]", "0", text)    # 数字を全て0にする
    return text

# 前処理＋単語分割
def tokenizer_with_preprocessing(text):
    return tokenizer_janome(preprocessing_text(text))

max_length = 25 # 1つの文章内の最大単語数
TEXT = torchtext.data.Field(
    sequential=True,    # 可変データか
    tokenize=tokenizer_with_preprocessing,  # 単語分割の関数
    use_vocab=True,     # 単語を辞書に追加するか
    lower=True,         # アルファベットを小文字にするか
    include_lengths=True,   # 単語数を保持するか
    batch_first=True,       # バッチの次元が先にくるか
    fix_length=max_length   # 各文章をパディングして同じ長さにする
)

LABEL = torchtext.data.Field(
    sequential=False,    # 可変データか
    use_vocab=False,     # 単語を辞書に追加するか
)

# Datasetの生成
# ※GoogleDriveに接続
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(path="./drive/My Drive/Transformer/data", train="text_train.tsv", validation="text_val.tsv", test="text_test.tsv",
                                     format="tsv", fields=[("Text", TEXT), ("Label", LABEL)])


     |████████████████████████████████| 21.5MB 69.3MB/s 


## 3.2. word2vec日本語学習済みモデルのダウンロード

In [ ]:
%cd ./drive/My\ Drive/Transformer/data
!curl -vLJO https://github.com/singletongue/WikiEntVec/releases/download/20190520/jawiki.all_vectors.200d.txt.bz2
!bunzip2 jawiki.all_vectors.200d.txt.bz2
%cd /content

In [ ]:
from gensim.models import KeyedVectors

# Word2Vec形式で保存
model = KeyedVectors.load_word2vec_format("./drive/My Drive/Transformer/data/jawiki.all_vectors.200d.txt", binary=False)
model.wv.save_word2vec_format("./drive/My Drive/Transformer/data/jawiki.all_vectors.200d.vec")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


In [ ]:
# torchtextで読込み
from torchtext.vocab import Vectors

japanese_word2vec_vectors = Vectors("./drive/My Drive/Transformer/data/jawiki.all_vectors.200d.vec")
print("1単語の次元数: {}".format(japanese_word2vec_vectors.dim))
print("単語数: {}".format(len(japanese_word2vec_vectors.itos)))

100%|█████████▉| 1511073/1511782 [01:51<00:00, 13238.90it/s]

AttributeError: ignored

In [ ]:
# 読み込んだ重みでボキャブラリーを生成
TEXT.build_vocab(train_ds, vectors=japanese_word2vec_vectors, min_freq=1)
print(TEXT.vocab.vectors.shape)
TEXT.vocab.vectors  # 単語のベクトルを表示

torch.Size([52, 200])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0320, -0.0681,  0.1761,  ...,  0.5876, -0.2313, -0.1415],
        ...,
        [ 0.3383, -0.1347, -0.1883,  ...,  0.4246,  0.0807, -0.2378],
        [ 0.0567, -0.0253,  0.2095,  ...,  0.2516, -0.1248, -0.0194],
        [ 0.7624,  0.0922,  0.2154,  ...,  0.2824, -0.1652,  0.1446]])

In [ ]:
TEXT.vocab.stoi     # 単語のIDを表示

In [ ]:
# ベクトルの足し算をしてみる
# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[42] - TEXT.vocab.vectors[39] + TEXT.vocab.vectors[48]

import torch.nn.functional as F
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[40], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[46], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[47], dim=0))
print("機械", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))

女王 tensor(0.5236)
王 tensor(0.5129)
王子 tensor(0.6787)
機械 tensor(0.1688)


## 3.3. fasttext日本語学習済みモデルのダウンロード

In [ ]:
%cd /content/drive/My\ Drive/Transformer/data
!curl -O https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.ja.vec
%cd /content

/content/drive/My Drive/Transformer/data
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1310M  100 1310M    0     0  10.2M      0  0:02:07  0:02:07 --:--:-- 11.4M
/content


In [ ]:
# torchtextで読込み
from torchtext.vocab import Vectors

japanese_fasttext_vectors = Vectors("./drive/My Drive/Transformer/data/wiki.ja.vec")
print("1単語の次元数: {}".format(japanese_fasttext_vectors.dim))
print("単語数: {}".format(len(japanese_fasttext_vectors.itos)))

In [ ]:
# 読み込んだ重みでボキャブラリーを生成
TEXT.build_vocab(train_ds, vectors=japanese_fasttext_vectors, min_freq=1)
print(TEXT.vocab.vectors.shape)
TEXT.vocab.vectors  # 単語のベクトルを表示

In [ ]:
TEXT.vocab.stoi     # 単語のIDを表示

In [ ]:
# ベクトルの足し算をしてみる
# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[42] - TEXT.vocab.vectors[39] + TEXT.vocab.vectors[48]

import torch.nn.functional as F
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[40], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[46], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[47], dim=0))
print("機械", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))

女王 tensor(0.9983)
王 tensor(0.9990)
王子 tensor(0.9983)
機械 tensor(0.9979)


## 3.4. fasttext英語学習済みモデルのダウンロード

In [ ]:
%cd /content/drive/My\ Drive/Transformer/data
!curl -O https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
!unzip wiki-news-300d-1M.vec.zip
!rm wiki-news-300d-1M.vec.zip
%cd /content

/content/drive/My Drive/Transformer/data
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  650M  100  650M    0     0  10.8M      0  0:00:59  0:00:59 --:--:-- 11.3M
Archive:  wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [ ]:
# torchtextで読込み
from torchtext.vocab import Vectors

english_fasttext_vectors = Vectors("./drive/My Drive/Transformer/data/wiki-news-300d-1M.vec")
print("1単語の次元数: {}".format(english_fasttext_vectors.dim))
print("単語数: {}".format(len(english_fasttext_vectors.itos)))

In [ ]:
english_fasttext_vectors["pen"]